## Создадим сервер

In [ ]:
!pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd
import dill

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 27LFNOH2h9iWspSJjtqQh9itwVo_7Fst6Ns6xk3QNMQ6UzxqV
!./ngrok http 80

--2022-04-18 13:37:16--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz.5’

ngrok-stable-linux- 100%[===================>]  13.13M  7.70MB/s    in 1.7s    

2022-04-18 13:37:18 (7.70 MB/s) - ‘ngrok-stable-linux-amd64.tgz.5’ saved [13770165/13770165]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Создаем сервис для обработки запросов к модели.

Загрузим обученную модель:

In [ ]:
with open('cardio_pipeline_lr.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

Создаем обработчик запросов и запускаем Flask:

In [ ]:
# Обработчики и запуск Flask
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"


@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}
    
    # ensure an image was properly uploaded to our endpoint
    age, gender, height, weight, ap_hi, ap_lo, cholesterol, gluc, smoke, alco, active = "", "", "", "", "", "", "", "", "", "", "" 
    request_json = request.get_json()
    
    if request_json["age"]:
        age = request_json['age']
    
    if request_json["gender"]:
        gender = request_json['gender']
                
    if request_json["height"]:
        height = request_json['height']
        
    if request_json["weight"]:
        weight = request_json['weight']
        
    if request_json["ap_hi"]:
        ap_hi = request_json['ap_hi']
        
    if request_json["ap_lo"]:
        ap_lo = request_json['ap_lo']
        
    if request_json["cholesterol"]:
        cholesterol = request_json['cholesterol']
        
    if request_json["gluc"]:
        gluc = request_json['gluc']
        
    if request_json["smoke"]:
        smoke = request_json['smoke']
        
    if request_json["alco"]:
        alco = request_json['alco']
        
    if request_json["active"]:
        active = request_json['active']
      
    preds = model.predict_proba(pd.DataFrame({"age": [age],
                                              "gender": [gender],
                                              "height": [height],
                                              "weight": [weight],
                                              "ap_hi": [ap_hi],
                                              "ap_lo": [ap_lo],
                                              "cholesterol": [cholesterol],
                                              "gluc": [gluc],
                                              "smoke": [smoke],
                                              "alco": [alco],
                                              "active": [active]}))
    data["predictions"] = preds[:, 1][0]
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a65f-34-75-114-17.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
